<a href="https://colab.research.google.com/github/Revolt234/AlgoritmoALS/blob/main/ProvaALS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from pyspark.sql import SparkSession

# Creare una sessione Spark
spark = SparkSession.builder.appName("MovieLens1MExample").getOrCreate()

# Caricare il dataset "ratings.dat" (sostituisci con il percorso del file)
file_path = "/content/drive/MyDrive/ratings1mln.dat"  # Cambia il percorso in base alla tua posizione del file
ratings = spark.read.csv(file_path, sep="::", header=False, inferSchema=True)

# Rinomina le colonne per maggiore chiarezza
ratings = ratings.withColumnRenamed("_c0", "userId") \
                 .withColumnRenamed("_c1", "movieId") \
                 .withColumnRenamed("_c2", "rating") \
                 .withColumnRenamed("_c3", "timestamp")

# Mostra alcune righe del dataset
ratings.show(5)


+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|   1193|     5|978300760|
|     1|    661|     3|978302109|
|     1|    914|     3|978301968|
|     1|   3408|     4|978300275|
|     1|   2355|     5|978824291|
+------+-------+------+---------+
only showing top 5 rows



In [ ]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import rand
# Dividere i dati in training e test
(training, test) = ratings.randomSplit([0.8, 0.2])

# Creare il modello ALS
als = ALS(
    maxIter=15,            # Numero massimo di iterazioni
    regParam=0.1,
    rank=50,
    userCol="userId",      # Colonna che rappresenta l'utente
    itemCol="movieId",     # Colonna che rappresenta il film
    ratingCol="rating",    # Colonna delle valutazioni
    coldStartStrategy="drop"  # Gestire i valori mancanti durante la previsione
)

# Addestrare il modello
model = als.fit(training)

# Predire le valutazioni sul set di test
predictions = model.transform(test)

# Valutare il modello con l'RMSE (Root Mean Squared Error)
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)

rmse = evaluator.evaluate(predictions)
print(f"RMSE del modello: {rmse}")


RMSE del modello: 0.8612467391265421


In [ ]:
# Mostrare 10 previsioni casuali
print("10 previsioni casuali:")
predictions.orderBy(rand()).show(10)

# Raccomandazioni per un sottoinsieme casuale di 10 utenti
print("Raccomandazioni per 10 utenti casuali:")
# Seleziona 10 utenti casuali
user_ids = ratings.select("userId").distinct().orderBy(rand()).limit(10)
user_recs = model.recommendForUserSubset(user_ids, 10)
user_recs.show(truncate=False)

# Raccomandazioni per un sottoinsieme casuale di 10 film
print("Raccomandazioni per 10 film casuali:")
# Seleziona 10 film casuali
movie_ids = ratings.select("movieId").distinct().orderBy(rand()).limit(10)
movie_recs = model.recommendForItemSubset(movie_ids, 10)
movie_recs.show(truncate=False)


10 previsioni casuali:
+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|  5971|   2396|     4|956954621| 3.9827843|
|   655|    417|     3|975704769|  2.859436|
|  4362|   2605|     2|965186336| 1.7482988|
|   423|   2692|     4|976281986|   2.46422|
|  3615|   1080|     5|966603734| 3.6770835|
|   151|   2657|     4|993119766| 3.3837156|
|   647|   1499|     2|975714036| 2.0937228|
|  4169|   1135|     3|971581056| 3.3493357|
|  1216|   2722|     2|974850112| 2.0061872|
|  5443|    426|     5|959980506| 3.3658001|
+------+-------+------+---------+----------+
only showing top 10 rows

Raccomandazioni per 10 utenti casuali:
+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userId|recommendations                                                                